In [1]:
%load_ext autoreload
%autoreload 2 

## Libraries

In [2]:
import os
import torch
import numpy as np
from data import *
from model import *
from train import *
from utils import * 
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, LambdaLR

## Getting a CUDA Device

In [3]:
device = get_device()
print(f"Current device: {device}")

Current device: cuda


## Training Dataset Specifications

In [4]:
augment=True
mode = 'Training'
preprocess = True
default_tranforms =  T.ToTensor()
augment_transforms=get_augment_transforms()
root_dir = os.path.join(os.getcwd(), 'dataset')
dataset_mean_file = os.path.join(os.getcwd(), 'mean_and_std.txt')

##  Dataset Instance

In [5]:
dataset = BrainTumorDataset(root_dir=root_dir, 
                            mode=mode,
                            preprocess=preprocess,
                            transforms=default_tranforms)

## Class Distribution of the Data before Augmentation

In [6]:
dataset.plot_data_distribution()

ValueError: object of too small depth for desired array

In [ ]:
f, axarr = plt.subplots(1,2)
axarr[0].imshow(dataset[220][0].permute(1, -1, 0))
axarr[1].imshow(dataset[10][0].permute(1, -1, 0))

## Test Dataset Specification

In [ ]:
test_mode='Testing'

In [ ]:
test_dataset = BrainTumorDataset(root_dir=root_dir, mode=test_mode,preprocess=preprocess,transforms=default_tranforms)

## Class Distribution of the Test Dataset 

In [ ]:
test_dataset.plot_data_distribution()

## Split Specifications

In [ ]:
num_samples = len(dataset)
train_samples, val_samples = split_samples(num_samples=num_samples, ratio=0.60)
test_samples = len(test_dataset)
print(f"num_samples: {num_samples}, \ntrain_samples : {train_samples},\nval_samples : {val_samples},\ntest_samples : {test_samples}")


In [ ]:
indecis = dataset.indecis().tolist()

In [ ]:
train_set= Subset(dataset, indecis[:train_samples], augment_transforms=augment_transforms)


In [ ]:
val_set= Subset(dataset, indecis[train_samples:])


In [ ]:
train_loader = DataLoader(train_set, batch_size = 128, shuffle=True, num_workers=4, collate_fn=None,pin_memory=True)
validation_loader = DataLoader(val_set, batch_size = 256, shuffle=True, num_workers=4, collate_fn=None,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = 256, shuffle=True, num_workers=4, collate_fn=None,pin_memory=True)

## Calculating the Mean and Standard Deviation of the Data Subset

In [ ]:
mean, std = get_mean_and_std(train_loader, train_samples, 256, dataset_mean_file, redo=False)

In [ ]:
print(mean, std)

## Set the Data Transforms to Apply Normalization

In [ ]:
new_transforms = get_transforms(mean=mean,std=std)

In [ ]:
dataset.set_transforms(new_transforms)
test_dataset.set_transforms(new_transforms)

## Images After Normalization

In [ ]:
f, axarr = plt.subplots(1,2)
axarr[0].imshow(train_set[3][0].permute(1, 2, 0))
axarr[1].imshow(test_dataset[3][0].permute(1, 2, 0))

In [ ]:
def my_collate(batch):
    imgs = [item[0] for item in batch]
    targets = torch.LongTensor([item[1] for item in batch])
    return (imgs, targets)

## A DataLoader Instance

In [ ]:
# train_loader = DataLoader(train_set, batch_size = 512, shuffle=True, num_workers=4, collate_fn=None,pin_memory=True)
# validation_loader = DataLoader(val_set, batch_size = 512, shuffle=True, num_workers=4, collate_fn=None,pin_memory=True)
# test_loader = DataLoader(test_dataset, batch_size = 512, shuffle=True, num_workers=4, collate_fn=None,pin_memory=True)

## The specifications of the BrainTumorClassifer Model

In [ ]:
in_channels = 3
num_classes = 4
bias=False

In [ ]:
model = BrainTumorClasssifer(in_channels=in_channels, num_classes=num_classes, bias=bias)

## Training Specifications

In [ ]:
learning_rate = 1e-3
weight_decay = 1e-2
momentum = 0.99
epochs=45

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay, amsgrad=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, amsgrad=True, betas=(0.9, 0.999))

In [ ]:
lr_lambda = custom_lr_factor

In [ ]:
scheduler = StepLR(optimizer, step_size=15, gamma=0.1)
# scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)

In [ ]:
weights = dataset.class_weights()

In [ ]:

criterion = torch.nn.CrossEntropyLoss(weight=None)

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
total_loss, epoch_times, accuracies_train, accuracies_validation = train(model, optimizer, train_loader, validation_loader, criterion, epochs,scheduler=scheduler, device=device)

## Accuracy & Loss Plots

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(10, 5), dpi=80)
plt.plot(range(epochs), accuracies_train, 'bo-')
plt.plot(range(epochs), accuracies_validation, 'ro-')
plt.legend(["Train Accuracy", "Validation Accuracy"], loc ="lower right")
plt.title('Training Accuracy VS Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy %")
plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(10, 4), dpi=80)
plt.plot(range(epochs), total_loss, 'bo-')
plt.legend(["Loss"], loc ="lower right")
plt.title('Total Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss ")
plt.show()

## Evaluating the Model on the Test Set

In [ ]:
from evaluate import *

In [ ]:
acc, loss = evaluate(model, test_loader, criterion, device=device)

In [ ]:
print(f"Testing accuracy: {acc}\nTesting loss: {loss}")

## Calculating and Plotting the Confusion Matrix

In [ ]:
y_true, y_pred = predictions(model=model, data_loader=test_loader, device=device)

In [ ]:
cf_matrix = calculate_confusion_matrix(y_true=y_true, y_pred=y_pred)

In [ ]:
plot_cf_matrix(cf_matrix, dataset.classes)

## Plot Class Accuracies

In [ ]:
plot_per_class_accuracy(cf_matrix, dataset.classes)

## Classification Report

In [ ]:
classification_report = get_classification_report(y_true=y_true,
                                                  y_pred=y_pred,
                                                  target_names=dataset.classes)

In [ ]:
print(classification_report)

In [ ]:
save_model(model, 'model_94_acc', os.getcwd())